Load Trained Weights

In [ ]:
model.load_weights('model.weights.h5')

Visualize Predictions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(5,3, figsize=(10,18))
j = np.random.randint(0, X_test.shape[0], 5)
for i in range(5):
    ax[i,0].imshow(X_test[j[i]], cmap='gray')
    ax[i,1].imshow(y_test[j[i]], cmap='gray')
    ax[i,0].set_title('Image')
    ax[i,1].set_title('Mask')
    ax[i,2].imshow(model.predict(np.expand_dims(X_test[j[i]],0),verbose=0)[0], cmap='gray')
    ax[i,2].set_title('Prediction')
fig.suptitle('Results', fontsize=16)
plt.show()

Plot ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

idx = np.random.randint(0, X_test.shape[0])
image = X_test[idx]
true_mask = y_test[idx]
pred_mask = model.predict(np.expand_dims(image, 0), verbose=0)[0]

y_true_flat = true_mask.flatten()
y_pred_flat = pred_mask.flatten()

fpr, tpr, _ = roc_curve((y_true_flat > 0.5).astype(int), y_pred_flat)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots(1, 4, figsize=(18, 4))

ax[0].imshow(image.squeeze(), cmap='gray')
ax[0].set_title('Input Image')
ax[0].axis('off')

ax[1].imshow(true_mask.squeeze(), cmap='gray')
ax[1].set_title('Ground Truth Mask')
ax[1].axis('off')

ax[2].imshow(pred_mask.squeeze(), cmap='gray')
ax[2].set_title('Predicted Mask')
ax[2].axis('off')

ax[3].plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.3f}')
ax[3].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax[3].set_xlim([0.0, 1.0])
ax[3].set_ylim([0.0, 1.05])
ax[3].set_xlabel('False Positive Rate')
ax[3].set_ylabel('True Positive Rate')
ax[3].set_title('ROC Curve')
ax[3].legend(loc='lower right')
ax[3].grid(True)

plt.tight_layout()
plt.show()

Plot Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Use previously predicted masks and threshold
# (ensure y_pred_thresholded is calculated before this)

y_true_flat = (y_test > 0.5).astype(int).flatten()
y_pred_flat = y_pred_thresholded.astype(int).flatten()

cm = confusion_matrix(y_true_flat, y_pred_flat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])

plt.figure(figsize=(5, 4))
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.grid(False)
plt.show()

Calculate and Display Evaluation Metrics

In [ ]:
from keras.metrics import MeanIoU, Precision, Recall
import pandas as pd

# Predictions
y_pred = model.predict(X_test, verbose=0)
y_pred_thresholded = y_pred > 0.5

# Metrics
IOU_keras = MeanIoU(num_classes=2)
IOU_keras.update_state(y_test, y_pred_thresholded)
iou = IOU_keras.result().numpy()

prec_score = Precision()
prec_score.update_state(y_test, y_pred_thresholded)
precision = prec_score.result().numpy()

recall_score = Recall()
recall_score.update_state(y_test, y_pred_thresholded)
recall = recall_score.result().numpy()

f1 = 2 * (precision * recall) / (precision + recall)

df = pd.DataFrame({
    'Metric': ['Mean IoU', 'Precision', 'Recall', 'F1 Score'],
    'Score': [iou, precision, recall, f1]
})

print('\033[96m' + '\nEvaluation Metrics:\n')
print(df.to_string(index=False, formatters={'Score': '{:.3f}'.format}))